# Immunity Algorithms

## AIRS (Artificial Immune Recognition System)

In [1]:
def random_vector(minmax)
  return Array.new(minmax.size) do |i|      
    minmax[i][0] + ((minmax[i][1] - minmax[i][0]) * rand())
  end
end

:random_vector

In [2]:
def generate_random_pattern(domain)  
  class_label = domain.keys[rand(domain.keys.size)]
  pattern = {:label=>class_label}
  pattern[:vector] = random_vector(domain[class_label])
  return pattern
end

:generate_random_pattern

In [3]:
def create_cell(vector, class_label)
  return {:label=>class_label, :vector=>vector}
end

:create_cell

In [4]:
def initialize_cells(domain)
  mem_cells = []
  domain.keys.each do |key|
    mem_cells << create_cell(random_vector([[0,1],[0,1]]), key)
  end
  return mem_cells
end

:initialize_cells

In [5]:
def distance(c1, c2)
  sum = 0.0
  c1.each_index {|i| sum += (c1[i]-c2[i])**2.0}  
  return Math.sqrt(sum)
end

:distance

In [6]:
def stimulate(cells, pattern)
  max_dist = distance([0.0,0.0], [1.0,1.0])
  cells.each do |cell|
    cell[:affinity] = distance(cell[:vector], pattern[:vector]) / max_dist
    cell[:stimulation] = 1.0 - cell[:affinity]
  end
end

:stimulate

In [7]:
def get_most_stimulated_cell(mem_cells, pattern)
  stimulate(mem_cells, pattern)
  return mem_cells.sort{|x,y| y[:stimulation] <=> x[:stimulation]}.first
end

:get_most_stimulated_cell

In [8]:
def mutate_cell(cell, best_match)
  range = 1.0 - best_match[:stimulation]
  cell[:vector].each_with_index do |v,i|
    min = [(v-(range/2.0)), 0.0].max
    max = [(v+(range/2.0)), 1.0].min
    cell[:vector][i] = min + (rand() * (max-min))
  end
  return cell
end

:mutate_cell

In [9]:
def create_arb_pool(pattern, best_match, clone_rate, mutate_rate)
  pool = []
  pool << create_cell(best_match[:vector], best_match[:label])
  num_clones = (best_match[:stimulation] * clone_rate * mutate_rate).round
  num_clones.times do 
    cell = create_cell(best_match[:vector], best_match[:label])
    pool << mutate_cell(cell, best_match)
  end
  return pool
end

:create_arb_pool

In [10]:
def competition_for_resources(pool, clone_rate, max_res)
  pool.each {|cell| cell[:resources] = cell[:stimulation] * clone_rate}
  pool.sort!{|x,y| x[:resources] <=> y[:resources]}
  total_resources = pool.inject(0.0){|sum,cell| sum + cell[:resources]}
  while total_resources > max_res
    cell = pool.delete_at(pool.size-1)
    total_resources -= cell[:resources]
  end
end

:competition_for_resources

In [19]:
def refine_arb_pool(pool, pattern, stim_thresh, clone_rate, max_res)  
  mean_stim, candidate = 0.0, nil
  begin
    stimulate(pool, pattern)
    candidate = pool.sort{|x,y| y[:stimulation] <=> x[:stimulation]}.first
    mean_stim = pool.inject(0.0){|s,c| s + c[:stimulation]} / pool.size
    if mean_stim < stim_thresh
      candidate = competition_for_resources(pool, clone_rate, max_res)
      pool.size.times do |i|
        cell = create_cell(pool[i][:vector], pool[i][:label])
        mutate_cell(cell, pool[i])
        pool << cell
      end
    end
  end until mean_stim >= stim_thresh   
  return candidate
end

:refine_arb_pool

In [20]:
def add_candidate_to_memory_pool(candidate, best_match, mem_cells)
  if candidate[:stimulation] > best_match[:stimulation]
    mem_cells << candidate
  end
end

:add_candidate_to_memory_pool

In [21]:
def classify_pattern(mem_cells, pattern)
  stimulate(mem_cells, pattern)
  return mem_cells.sort{|x,y| y[:stimulation] <=> x[:stimulation]}.first
end

:classify_pattern

In [26]:
def train_system(mem_cells, domain, num_patterns, clone_rate, mutate_rate, stim_thresh, max_res)
  num_patterns.times do |i|
    pattern = generate_random_pattern(domain)
    best_match = get_most_stimulated_cell(mem_cells, pattern)
    if best_match[:label] != pattern[:label]
      mem_cells << create_cell(pattern[:vector], pattern[:label])
    elsif best_match[:stimulation] < 1.0
      pool = create_arb_pool(pattern, best_match, clone_rate, mutate_rate)
      cand = refine_arb_pool(pool,pattern, stim_thresh, clone_rate, max_res)
      add_candidate_to_memory_pool(cand, best_match, mem_cells)
    end
    #puts " > iter=#{i+1}, mem_cells=#{mem_cells.size}"
  end
end

:train_system

In [27]:
def test_system(mem_cells, domain, num_trials=50)
  correct = 0
  num_trials.times do 
    pattern = generate_random_pattern(domain)
    best = classify_pattern(mem_cells, pattern)
    correct += 1 if best[:label] == pattern[:label]
  end
  #puts "Finished test with a score of #{correct}/#{num_trials}"
  return correct
end

:test_system

In [28]:
def execute(domain, num_patterns, clone_rate, mutate_rate, stim_thresh, max_res)  
  mem_cells = initialize_cells(domain)
  train_system(mem_cells, domain, num_patterns, clone_rate, mutate_rate, stim_thresh, max_res)
  test_system(mem_cells, domain)
  return mem_cells
end

:execute

In [30]:
domain = {"A"=>[[0,0.4999999],[0,0.4999999]],"B"=>[[0.5,1],[0.5,1]]}
num_patterns = 50

# algorithm configuration
clone_rate = 10
mutate_rate = 2.0
stim_thresh = 0.9
max_res = 150

# execute the algorithm
puts = execute(domain, num_patterns, clone_rate, mutate_rate, stim_thresh, max_res)


[{:label=>"A", :vector=>[0.20004063852829315, 0.22211153063858124], :affinity=>0.16067141553984673, :stimulation=>0.8393285844601532}, {:label=>"B", :vector=>[0.9577024063285609, 0.7508360728248049], :affinity=>0.6784435838737568, :stimulation=>0.3215564161262432}, {:label=>"B", :vector=>[0.9577024063285609, 0.7508360728248049], :affinity=>0.6784435838737568, :stimulation=>0.3215564161262432, :resources=>6.907255660453178}, {:label=>"B", :vector=>[0.9577024063285609, 0.7508360728248049], :affinity=>0.6784435838737568, :stimulation=>0.3215564161262432, :resources=>8.54496984390298}, {:label=>"A", :vector=>[0.20004063852829315, 0.22211153063858124], :affinity=>0.16067141553984673, :stimulation=>0.8393285844601532, :resources=>7.954637082527562}, {:label=>"A", :vector=>[0.20004063852829315, 0.22211153063858124], :affinity=>0.16067141553984673, :stimulation=>0.8393285844601532}, {:label=>"B", :vector=>[0.9577024063285609, 0.7508360728248049], :affinity=>0.6784435838737568, :stimulation=>0.

## Clonal Selection Algorithm (CLONALG)

In [31]:
def objective_function(vector)
  return vector.inject(0.0) {|sum, x| sum + (x**2.0)}
end

:objective_function

In [32]:
def decode(bitstring, search_space, bits_per_param)
  vector = []
  search_space.each_with_index do |bounds, i|
    off, sum = i*bits_per_param, 0.0
    param = bitstring[off...(off+bits_per_param)].reverse
    param.size.times do |j|
      sum += ((param[j].chr=='1') ? 1.0 : 0.0) * (2.0 ** j.to_f)
    end
    min, max = bounds
    vector << min + ((max-min)/((2.0**bits_per_param.to_f)-1.0)) * sum
  end
  return vector
end

:decode

In [33]:
def evaluate(pop, search_space, bits_per_param)
  pop.each do |p|
    p[:vector] = decode(p[:bitstring], search_space, bits_per_param)
    p[:cost] = objective_function(p[:vector])
  end
end

:evaluate

In [34]:
def random_bitstring(num_bits)
  return (0...num_bits).inject(""){|s,i| s<<((rand<0.5) ? "1" : "0")}
end

:random_bitstring

In [35]:
def point_mutation(bitstring, rate)
  child = ""
   bitstring.size.times do |i|
     bit = bitstring[i].chr
     child << ((rand()<rate) ? ((bit=='1') ? "0" : "1") : bit)
  end
  return child
end

:point_mutation

In [36]:
def calculate_mutation_rate(antibody, mutate_factor=-2.5)
  return Math.exp(mutate_factor * antibody[:affinity])
end

:calculate_mutation_rate

In [37]:
def num_clones(pop_size, clone_factor)
  return (pop_size * clone_factor).floor
end

:num_clones

In [38]:
def calculate_affinity(pop)
  pop.sort!{|x,y| x[:cost]<=>y[:cost]}
  range = pop.last[:cost] - pop.first[:cost]
  if range == 0.0
    pop.each {|p| p[:affinity] = 1.0}
  else
    pop.each {|p| p[:affinity] = 1.0-(p[:cost]/range)}
  end
end

:calculate_affinity

In [39]:
def clone_and_hypermutate(pop, clone_factor)
  clones = []
  num_clones = num_clones(pop.size, clone_factor)
  calculate_affinity(pop)
  pop.each do |antibody|
    m_rate = calculate_mutation_rate(antibody)
    num_clones.times do 
      clone = {}
      clone[:bitstring] = point_mutation(antibody[:bitstring], m_rate)
      clones << clone
    end
  end
  return clones  
end

:clone_and_hypermutate

In [40]:
def random_insertion(search_space, pop, num_rand, bits_per_param)
  return pop if num_rand == 0
  rands = Array.new(num_rand) do |i|
    {:bitstring=>random_bitstring(search_space.size*bits_per_param)}
  end
  evaluate(rands, search_space, bits_per_param)
  return (pop+rands).sort{|x,y| x[:cost]<=>y[:cost]}.first(pop.size)
end

:random_insertion

In [41]:
def search(search_space, max_gens, pop_size, clone_factor, num_rand, bits_per_param=16)
  pop = Array.new(pop_size) do |i|
    {:bitstring=>random_bitstring(search_space.size*bits_per_param)}
  end
  evaluate(pop, search_space, bits_per_param)
  best = pop.min{|x,y| x[:cost]<=>y[:cost]}
  max_gens.times do |gen|
    clones = clone_and_hypermutate(pop, clone_factor)
    evaluate(clones, search_space, bits_per_param)
    pop = (pop+clones).sort{|x,y| x[:cost]<=>y[:cost]}.first(pop_size)
    pop = random_insertion(search_space, pop, num_rand, bits_per_param)
    best = (pop + [best]).min{|x,y| x[:cost]<=>y[:cost]}
    #puts " > gen #{gen+1}, f=#{best[:cost]}, s=#{best[:vector].inspect}"
  end  
  return best
end

:search

In [42]:
# problem configuration
problem_size = 2
search_space = Array.new(problem_size) {|i| [-5, +5]}

# algorithm configuration
max_gens = 100
pop_size = 100
clone_factor = 0.1
num_rand = 2

# execute the algorithm
puts search(search_space, max_gens, pop_size, clone_factor, num_rand)
#puts "done! Solution: f=#{best[:cost]}, s=#{best[:vector].inspect}"

{:bitstring=>"10000000000000001000000000000000", :vector=>[7.629510948348184e-05, 7.629510948348184e-05], :cost=>1.164188746219296e-08, :affinity=>1.0}


## Dendritic Cell Algorithm

In [43]:
def rand_in_bounds(min, max)
  return  min + ((max-min) * rand()) 
end

:rand_in_bounds

In [44]:
def random_vector(search_space)
  return Array.new(search_space.size) do |i|   
    rand_in_bounds(search_space[i][0], search_space[i][1])
  end
end

:random_vector

In [45]:
def construct_pattern(class_label, domain, p_safe, p_danger)
  set = domain[class_label]
  selection = rand(set.size)
  pattern = {}
  pattern[:class_label] = class_label
  pattern[:input] = set[selection]
  pattern[:safe] = (rand() * p_safe * 100)
  pattern[:danger] = (rand() * p_danger * 100)  
  return pattern
end

:construct_pattern

In [62]:
def generate_pattern(domain, p_anomaly, p_normal, prob_create_anom=0.5)
  pattern = nil  
  if rand() < prob_create_anom
    pattern = construct_pattern("Anomaly", domain, 1.0-p_normal, p_anomaly)
    #puts ">Generated Anomaly [#{pattern[:input]}]"
  else
    pattern = construct_pattern("Normal", domain, p_normal, 1.0-p_anomaly)
  end
  return pattern
end

:generate_pattern

In [63]:
def initialize_cell(thresh, cell={})
  cell[:lifespan] = 1000.0
  cell[:k] = 0.0
  cell[:cms] = 0.0
  cell[:migration_threshold] = rand_in_bounds(thresh[0], thresh[1])
  cell[:antigen] = {}
  return cell
end

:initialize_cell

In [64]:
def store_antigen(cell, input)
  if cell[:antigen][input].nil?
    cell[:antigen][input] = 1
  else
    cell[:antigen][input] += 1
  end
end

:store_antigen

In [65]:
def expose_cell(cell, cms, k, pattern, threshold)
  cell[:cms] += cms
  cell[:k] += k
  cell[:lifespan] -= cms 
  store_antigen(cell, pattern[:input]) 
  initialize_cell(threshold, cell) if cell[:lifespan] <= 0
end

:expose_cell

In [66]:
def can_cell_migrate?(cell)
  return (cell[:cms]>=cell[:migration_threshold] and !cell[:antigen].empty?)
end

:can_cell_migrate?

In [67]:
def expose_all_cells(cells, pattern, threshold)
  migrate = []
  cms = (pattern[:safe] + pattern[:danger]) 
  k = pattern[:danger] - (pattern[:safe] * 2.0)  
  cells.each do |cell|
    expose_cell(cell, cms, k, pattern, threshold)
    if can_cell_migrate?(cell)
      migrate << cell
      cell[:class_label] = (cell[:k]>0) ? "Anomaly" : "Normal"
    end
  end
  return migrate
end

:expose_all_cells

In [68]:
def train_system(domain, max_iter, num_cells, p_anomaly, p_normal, thresh)
  immature_cells = Array.new(num_cells){ initialize_cell(thresh) }
  migrated = []
  max_iter.times do |iter|
    pattern = generate_pattern(domain, p_anomaly, p_normal)
    migrants = expose_all_cells(immature_cells, pattern, thresh)
    migrants.each do |cell|
      immature_cells.delete(cell)
      immature_cells << initialize_cell(thresh)
      migrated << cell
    end
    #puts "> iter=#{iter} new=#{migrants.size}, migrated=#{migrated.size}"
  end
  return migrated
end

:train_system

In [69]:
def classify_pattern(migrated, pattern)
  input = pattern[:input]
  num_cells, num_antigen = 0, 0
  migrated.each do |cell|
    if cell[:class_label] == "Anomaly" and !cell[:antigen][input].nil?
      num_cells += 1
      num_antigen += cell[:antigen][input]
    end
  end
  mcav = num_cells.to_f / num_antigen.to_f
  return (mcav>0.5) ? "Anomaly" : "Normal"
end

:classify_pattern

In [70]:
def test_system(migrated, domain, p_anomaly, p_normal, num_trial=100)
  correct_norm = 0
  num_trial.times do
    pattern = construct_pattern("Normal", domain, p_normal, 1.0-p_anomaly)
    class_label = classify_pattern(migrated, pattern)
    correct_norm += 1 if class_label == "Normal"
  end
  puts "Finished testing Normal inputs #{correct_norm}/#{num_trial}"
  correct_anom = 0
  
  num_trial.times do
    pattern = construct_pattern("Anomaly", domain, 1.0-p_normal, p_anomaly)
    class_label = classify_pattern(migrated, pattern)
    correct_anom += 1 if class_label == "Anomaly"
  end
  
  puts "Finished testing Anomaly inputs #{correct_anom}/#{num_trial}"
  return [correct_norm, correct_anom]
end

:test_system

In [71]:
def execute(domain, max_iter, num_cells, p_anom, p_norm, thresh)  
  migrated=train_system(domain, max_iter, num_cells, p_anom, p_norm, thresh)
  test_system(migrated, domain, p_anom, p_norm)
  return migrated
end

:execute

In [72]:
# problem configuration
domain = {}
domain["Normal"]  = Array.new(50){|i| i}
domain["Anomaly"] = Array.new(5){|i| (i+1)*10}
domain["Normal"]  = domain["Normal"] - domain["Anomaly"]

p_anomaly = 0.70
p_normal = 0.95

# algorithm configuration
iterations = 100
num_cells = 10
thresh = [5,15]

# execute the algorithm
execute(domain, iterations, num_cells, p_anomaly, p_normal, thresh)

Finished testing Normal inputs 82/100
Finished testing Anomaly inputs 100/100


[{:lifespan=>908.3839650448392, :k=>-117.06794335296381, :cms=>91.61603495516081, :migration_threshold=>8.7544536032068, :antigen=>{12=>1}, :class_label=>"Normal"}, {:lifespan=>908.3839650448392, :k=>-117.06794335296381, :cms=>91.61603495516081, :migration_threshold=>6.912436629065645, :antigen=>{12=>1}, :class_label=>"Normal"}, {:lifespan=>908.3839650448392, :k=>-117.06794335296381, :cms=>91.61603495516081, :migration_threshold=>14.582974802490437, :antigen=>{12=>1}, :class_label=>"Normal"}, {:lifespan=>908.3839650448392, :k=>-117.06794335296381, :cms=>91.61603495516081, :migration_threshold=>7.669333842753582, :antigen=>{12=>1}, :class_label=>"Normal"}, {:lifespan=>908.3839650448392, :k=>-117.06794335296381, :cms=>91.61603495516081, :migration_threshold=>14.803957121843066, :antigen=>{12=>1}, :class_label=>"Normal"}, {:lifespan=>908.3839650448392, :k=>-117.06794335296381, :cms=>91.61603495516081, :migration_threshold=>6.804102437823769, :antigen=>{12=>1}, :class_label=>"Normal"}, {:l

## Negative Selection Algorithm

In [73]:
def random_vector(minmax)
  return Array.new(minmax.length) do |i|      
    minmax[i][0] + ((minmax[i][1] - minmax[i][0]) * rand())
  end
end

:random_vector

In [74]:
def euclidean_distance(c1, c2)
  sum = 0.0
  c1.each_index {|i| sum += (c1[i]-c2[i])**2.0}  
  return Math.sqrt(sum)
end

:euclidean_distance

In [75]:
def contains?(vector, space)
  vector.each_with_index do |v,i|
    return false if v<space[i][0] or v>space[i][1]
  end
  return true
end

:contains?

In [76]:
def matches?(vector, dataset, min_dist)
  dataset.each do |pattern|
    dist = euclidean_distance(vector, pattern[:vector])
    return true if dist <= min_dist
  end
  return false
end

:matches?

In [77]:
def generate_detectors(max_detectors, search_space, self_dataset, min_dist)
  detectors = []
  begin
    detector = {:vector=>random_vector(search_space)}
    if !matches?(detector[:vector], self_dataset, min_dist)
      detectors << detector if !matches?(detector[:vector], detectors, 0.0)
    end
  end while detectors.size < max_detectors
  return detectors
end

:generate_detectors

In [78]:
def generate_self_dataset(num_records, self_space, search_space)
  self_dataset = []
  begin
    pattern = {}
    pattern[:vector] = random_vector(search_space)
    next if matches?(pattern[:vector], self_dataset, 0.0)
    if contains?(pattern[:vector], self_space)
      self_dataset << pattern 
    end
  end while self_dataset.length < num_records
  return self_dataset
end

:generate_self_dataset

In [82]:
def apply_detectors(detectors, bounds, self_dataset, min_dist, trials=50)
  correct = 0
  trials.times do |i|
    input = {:vector=>random_vector(bounds)}
    actual = matches?(input[:vector], detectors, min_dist) ? "N" : "S"
    expected = matches?(input[:vector], self_dataset, min_dist) ? "S" : "N"
    correct += 1 if actual==expected
    #puts "#{i+1}/#{trials}: predicted=#{actual}, expected=#{expected}"
  end
  #puts "Done. Result: #{correct}/#{trials}"
  return correct
end

:apply_detectors

In [83]:
def execute(bounds, self_space, max_detect, max_self, min_dist)
  
  self_dataset = generate_self_dataset(max_self, self_space, bounds)
  puts "Done: prepared #{self_dataset.size} self patterns."
  
  detectors = generate_detectors(max_detect, bounds, self_dataset, min_dist)
  puts "Done: prepared #{detectors.size} detectors."
  
  apply_detectors(detectors, bounds, self_dataset, min_dist)
  return detectors
end

:execute

In [84]:
# problem configuration
problem_size = 2
search_space = Array.new(problem_size) {[0.0, 1.0]}
self_space = Array.new(problem_size) {[0.5, 1.0]}
max_self = 150

# algorithm configuration
max_detectors = 300
min_dist = 0.05

# execute the algorithm
puts execute(search_space, self_space, max_detectors, max_self, min_dist)

Done: prepared 150 self patterns.
Done: prepared 300 detectors.
[{:vector=>[0.38130294149737476, 0.7510466049171602]}, {:vector=>[0.06558152177658316, 0.2422765351104924]}, {:vector=>[0.9448919060068979, 0.10285701281612958]}, {:vector=>[0.4780169177216149, 0.033509599041656735]}, {:vector=>[0.5155111316989786, 0.030428615789059976]}, {:vector=>[0.14568325725726838, 0.4857016672608099]}, {:vector=>[0.37972681568648836, 0.15679638594086076]}, {:vector=>[0.32491094164950485, 0.43908850041015657]}, {:vector=>[0.23139940388571245, 0.9637870079445124]}, {:vector=>[0.3219243579327915, 0.7188150165798685]}, {:vector=>[0.031293852949893286, 0.3414701662041404]}, {:vector=>[0.865843863347512, 0.0806975274587185]}, {:vector=>[0.3255277955769905, 0.36248102122363446]}, {:vector=>[0.38704737111014353, 0.692532588094816]}, {:vector=>[0.4225830535776841, 0.38033790866267503]}, {:vector=>[0.20943864390640343, 0.5732552886178811]}, {:vector=>[0.4683143118916492, 0.8532390388181508]}, {:vector=>[0.5662

## Optimization Artificial Immune Network (opt-aiNet)

In [85]:
def objective_function(vector)
  return vector.inject(0.0) {|sum, x| sum + (x**2.0)}
end

:objective_function

In [86]:
def random_vector(minmax)
  return Array.new(minmax.size) do |i|      
    minmax[i][0] + ((minmax[i][1] - minmax[i][0]) * rand())
  end
end

:random_vector

In [87]:
def random_gaussian(mean=0.0, stdev=1.0)
  u1 = u2 = w = 0
  begin
    u1 = 2 * rand() - 1
    u2 = 2 * rand() - 1
    w = u1 * u1 + u2 * u2
  end while w >= 1
  w = Math.sqrt((-2.0 * Math.log(w)) / w)
  return mean + (u2 * w) * stdev
end

:random_gaussian

In [88]:
def clone(parent)
  v = Array.new(parent[:vector].size) {|i| parent[:vector][i]}
  return {:vector=>v}
end

:clone

In [89]:
def mutation_rate(beta, normalized_cost)
  return (1.0/beta) * Math.exp(-normalized_cost)
end

:mutation_rate

In [90]:
def mutate(beta, child, normalized_cost)
  child[:vector].each_with_index do |v, i|
    alpha = mutation_rate(beta, normalized_cost)
    child[:vector][i] = v + alpha * random_gaussian()
  end
end

:mutate

In [91]:
def clone_cell(beta, num_clones, parent)
  clones = Array.new(num_clones) {clone(parent)}
  clones.each {|clone| mutate(beta, clone, parent[:norm_cost])}
  clones.each{|c| c[:cost] = objective_function(c[:vector])}
  clones.sort!{|x,y| x[:cost] <=> y[:cost]}
  return clones.first
end

:clone_cell

In [92]:
def calculate_normalized_cost(pop)
  pop.sort!{|x,y| x[:cost]<=>y[:cost]}
  range = pop.last[:cost] - pop.first[:cost]

  if range == 0.0
    pop.each {|p| p[:norm_cost] = 1.0}
  else
    pop.each {|p| p[:norm_cost] = 1.0-(p[:cost]/range)}
  end
end

:calculate_normalized_cost

In [93]:
def average_cost(pop)
  sum = pop.inject(0.0){|sum,x| sum + x[:cost]}
  return sum / pop.size.to_f
end

:average_cost

In [94]:
def distance(c1, c2)
  sum = 0.0
  c1.each_index {|i| sum += (c1[i]-c2[i])**2.0}
  return Math.sqrt(sum)
end

:distance

In [95]:
def get_neighborhood(cell, pop, aff_thresh)
  neighbors = []
  pop.each do |p|
    neighbors << p if distance(p[:vector], cell[:vector]) < aff_thresh
  end
  return neighbors
end

:get_neighborhood

In [96]:
def affinity_supress(population, aff_thresh)
  pop = []
  population.each do |cell|
    neighbors = get_neighborhood(cell, population, aff_thresh)
    neighbors.sort!{|x,y| x[:cost] <=> y[:cost]}
    pop << cell if neighbors.empty? or cell.equal?(neighbors.first)
  end  
  return pop
end

:affinity_supress

In [97]:
def search(search_space, max_gens, pop_size, num_clones, beta, num_rand, aff_thresh)
  pop = Array.new(pop_size) {|i| {:vector=>random_vector(search_space)} }
  pop.each{|c| c[:cost] = objective_function(c[:vector])}
  best = nil
  max_gens.times do |gen|
    pop.each{|c| c[:cost] = objective_function(c[:vector])}
    calculate_normalized_cost(pop)
    pop.sort!{|x,y| x[:cost] <=> y[:cost]}
    best = pop.first if best.nil? or pop.first[:cost] < best[:cost]
    avgCost, progeny = average_cost(pop), nil
    begin
      progeny=Array.new(pop.size){|i| clone_cell(beta, num_clones, pop[i])}
    end until average_cost(progeny) < avgCost
    pop = affinity_supress(progeny, aff_thresh)
    num_rand.times {pop << {:vector=>random_vector(search_space)}} 
    #puts " > gen #{gen+1}, popSize=#{pop.size}, fitness=#{best[:cost]}"
  end
  return best
end

:search

In [98]:
# problem configuration
problem_size = 2
search_space = Array.new(problem_size) {|i| [-5, +5]}

# algorithm configuration
max_gens = 150
pop_size = 20
num_clones = 10
beta = 100
num_rand = 2
aff_thresh = (search_space[0][1]-search_space[0][0])*0.05

# execute the algorithm
puts search(search_space, max_gens, pop_size, num_clones, beta, num_rand, aff_thresh)
#puts "done! Solution: f=#{best[:cost]}, s=#{best[:vector].inspect}"

{:vector=>[0.00035146881409522235, 0.00013547195123755993], :cost=>1.4188297685361378e-07, :norm_cost=>0.9999999959104384}
